In [2]:
"""
example_full.py

Demonstrates the complete workflow:
  1. Generate a random convex polygon and fixed interior seeds.
  2. Plot the Voronoi diagram for the fixed seeds.
  3. Define feasibility thresholds and build the problem specification.
  4. Configure solver parameters for various algorithms.
  5. Run each solver to maximize additional points, with optional intermediate plots.
"""

import random
from random_polygon import generate_random_convex_polygon_with_fixed
from losses import loss_type_1
from find_max_additional import find_max_additional
from plot_voronoi import plot_voronoi


def main():
    # ────────────────────────────────────────────────────────────────
    # 1) Generate polygon + fixed seeds
    # ────────────────────────────────────────────────────────────────
    # Use a fixed RNG seed for reproducibility.
    POLY_SEED = 43
    HULL_SAMPLES = 50
    NUM_FIXED = 6

    polygon, fixed = generate_random_convex_polygon_with_fixed(
        rand_seed=POLY_SEED,
        convexhullparam=HULL_SAMPLES,
        numpoints=NUM_FIXED
    )

    # ────────────────────────────────────────────────────────────────
    # 2) Initial Voronoi plot (fixed seeds only)
    # ────────────────────────────────────────────────────────────────
    print("Plotting initial Voronoi (fixed seeds only).")
    plot_voronoi(fixed, [], polygon, solver="—")

    # ────────────────────────────────────────────────────────────────
    # 3) Feasibility thresholds
    # ────────────────────────────────────────────────────────────────
    A_MIN = 0.03   # required minimum Voronoi cell area
    L2_MIN = 0.10  # required minimum distance between any two seeds

    # Problem specification passed to solvers
    problem = {
        "fixed":   fixed,
        "polygon": polygon,
        "A_min":   A_MIN,
        "l2_min":  L2_MIN,
        "loss":    lambda pts: loss_type_1(pts, polygon, L2_MIN)
    }

    # ────────────────────────────────────────────────────────────────
    # 4) Solver parameter configurations
    # ────────────────────────────────────────────────────────────────
    common_params = {
        "popsize": 40,
        "maxiter": 200,
        "alpha":   0.75,
        "seed":    123
    }

    ga_params = {
        **common_params,
        "ngen": 100,
        "sigma": 0.1,
        "indpb": 0.1,
        "cxpb":  0.5,
        "mutpb": 0.2
    }

    de_params = {
        "popsize": 15,   # multiplier for DE
        "maxiter": 200,
        "seed":    123
    }

    spsa_params = {
        "a":        0.1,
        "c":        0.1,
        "alpha":    0.602,
        "gamma":    0.101,
        "maxiter":  200,
        "restarts": 5,
        "seed":     123
    }

    # List of (solver_name, params_dict) to run
    solvers_to_run = [
        ("spsa", spsa_params),
        ("qpso", common_params),
        ("qpso_pairwise", common_params),
        ("ga", ga_params),
        ("de", de_params),
    ]

    # ────────────────────────────────────────────────────────────────
    # 5) Execute each solver
    # ────────────────────────────────────────────────────────────────
    for name, params in solvers_to_run:
        print(f"\n=== Running solver: {name} ===")
        config = {"name": name, "params": params}

        # plot_each=True will show intermediate feasible configurations
        max_count, best_added = find_max_additional(
            problem, config, plot_each=True
        )

        print(f"Solver {name} → Max extra points: {max_count}")
        print(f"Solver {name} → Best added points: {best_added}")


if __name__ == "__main__":
    main()


TypeError: generate_random_convex_polygon_with_fixed() got an unexpected keyword argument 'convexhullparam'